In [1]:

import pandas as pd
import plotly.express as px


In [4]:

# Cargar datos
homicidios_long = pd.read_csv("../data/processed/homicidios_canton_long.csv")
femicidios_long = pd.read_csv("../data/processed/femicidios_canton_long.csv")
denuncias_long = pd.read_csv("../data/processed/denuncias_prov_long.csv")


### 📊 Evolución mensual de homicidios

In [5]:

fig = px.line(homicidios_long.groupby('fecha')['homicidios'].sum().reset_index(),
              x='fecha', y='homicidios', title='Evolución mensual de homicidios en Ecuador')
fig.show()


### 📊 Evolución mensual de femicidios

In [6]:

fig = px.line(femicidios_long.groupby('fecha')['femicidios'].sum().reset_index(),
              x='fecha', y='femicidios', title='Evolución mensual de femicidios en Ecuador')
fig.show()


### 📊 Denuncias por tipo de delito

In [7]:

fig = px.line(denuncias_long.groupby(['fecha', 'delitos_de_mayor_incidencia'])['denuncias'].sum().reset_index(),
              x='fecha', y='denuncias', color='delitos_de_mayor_incidencia',
              title='Evolución mensual de denuncias por tipo de delito')
fig.show()


### 📊 Comparación anual de homicidios, femicidios y denuncias

In [13]:
# Verificar tipo de dato antes de aplicar .dt
homicidios_long['fecha'] = pd.to_datetime(homicidios_long['fecha'], errors='coerce')
femicidios_long['fecha'] = pd.to_datetime(femicidios_long['fecha'], errors='coerce')
denuncias_long['fecha'] = pd.to_datetime(denuncias_long['fecha'], errors='coerce')

# Eliminar filas con fechas inválidas
homicidios_long = homicidios_long.dropna(subset=['fecha'])
femicidios_long = femicidios_long.dropna(subset=['fecha'])
denuncias_long = denuncias_long.dropna(subset=['fecha'])


homicidios_long['año'] = homicidios_long['fecha'].dt.year
femicidios_long['año'] = femicidios_long['fecha'].dt.year
denuncias_long['año'] = denuncias_long['fecha'].dt.year

# Paso 1: Agregar homicidios y femicidios
agg_homicidios = homicidios_long.groupby('año')['homicidios'].sum().reset_index()
agg_homicidios['tipo'] = 'Homicidios'
agg_homicidios.rename(columns={'homicidios': 'cantidad'}, inplace=True)

agg_femicidios = femicidios_long.groupby('año')['femicidios'].sum().reset_index()
agg_femicidios['tipo'] = 'Femicidios'
agg_femicidios.rename(columns={'femicidios': 'cantidad'}, inplace=True)

# Paso 2: Agregar denuncias por tipo
agg_denuncias = (
    denuncias_long
    .groupby(['año', 'delitos_de_mayor_incidencia'])['denuncias']
    .sum()
    .reset_index()
    .rename(columns={'delitos_de_mayor_incidencia': 'tipo', 'denuncias': 'cantidad'})
)

# Paso 3: Unir todo
df_combined = pd.concat([agg_homicidios, agg_femicidios, agg_denuncias], ignore_index=True)

# Paso 4: Visualizar con Plotly
import plotly.express as px

fig = px.bar(df_combined,
             x='año',
             y='cantidad',
             color='tipo',
             barmode='group',
             title='Comparación anual de delitos y tipos de denuncias')

fig.update_layout(xaxis_title='Año', yaxis_title='Cantidad de delitos')
fig.show()


### Delitos más frecuentes (portipo de denuncia)

In [18]:
# Agrupar todas las denuncias por tipo (acumulado total)
top_delitos = (
    denuncias_long
    .groupby('delitos_de_mayor_incidencia')['denuncias']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# Visualizar con Plotly
fig = px.bar(top_delitos,
             x='denuncias',
             y='delitos_de_mayor_incidencia',
             orientation='h',
             title='Top 10 de delitos más denunciados en Ecuador',
             labels={'delitos_de_mayor_incidencia': 'Tipo de delito', 'denuncias': 'Cantidad'})
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


### Provincias con más homicidios, femicidios y denuncias

In [20]:
# Excluir Totales Nacionales
homicidios_prov = homicidios_long[homicidios_long['provincia'] != 'TOTAL NACIONAL']
femicidios_prov = femicidios_long[femicidios_long['provincia'] != 'TOTAL NACIONAL']
denuncias_prov = denuncias_long[denuncias_long['provincia'] != 'NACIONAL']

# Agrupar por provincia
homicidios_top = (
    homicidios_prov
    .groupby('provincia')['homicidios']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

femicidios_top = (
    femicidios_prov
    .groupby('provincia')['femicidios']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

denuncias_top = (
    denuncias_prov
    .groupby('provincia')['denuncias']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

# Gráficos
fig1 = px.bar(homicidios_top, x='provincia', y='homicidios',
              title='Top 10 provincias con más homicidios')
fig2 = px.bar(femicidios_top, x='provincia', y='femicidios',
              title='Top 10 provincias con más femicidios')
fig3 = px.bar(denuncias_top, x='provincia', y='denuncias',
              title='Top 10 provincias con más denuncias registradas')

fig1.show()
fig2.show()
fig3.show()